# Week 3 Application Assignment

***

## Import Libraries

In [1]:
import numpy as np
from numpy import count_nonzero
from numpy import median
from numpy import mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

import datetime
from datetime import datetime, timedelta

import eli5
from IPython.display import display

#import os
#import zipfile
import scipy.stats
from collections import Counter

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Lasso, Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, auc, classification_report, confusion_matrix, f1_score
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')



# Use Feature-Engine library
#import feature_engine
#from feature_engine import imputation as mdi
#from feature_engine.outlier_removers import Winsorizer
#from feature_engine import categorical_encoders as ce
#from feature_engine.discretisation import EqualWidthDiscretiser, EqualFrequencyDiscretiser
#from feature_engine.discretisation import ArbitraryDiscretiser, DecisionTreeDiscretiser
#from feature_engine.encoding import OrdinalEncoder

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


## Exploratory Data Analysis

The dataset also contains a column IndustryType, which is created based on the column Industry in the raw data. Note that Industry has many categories. The analyst who prepared the data chose to combine some categories, which resulted in the column IndustryType. IndustryType has five categories: Department, Discount, Grocery, Restaurants, Specialty. You can create a set of dummy variables based on IndustryType in XLMiner by using the Transform functions.

In [2]:
df = pd.read_csv("crp_cleandata.csv")

In [3]:
df

,Retailer,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Industry,Reward,ProgramName,RewardType,RewardStructure,RewardSize,ExpirationMonth,IndustryType
0,A&P,74,5.83,5.83,48.85,0.28,"Discount, Variety Stores",0,No rewards program,-,-,-,-,Discount
1,Albertsons,21,19.45,19.45,69.02,1.02,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
2,Aldi,38,10.90,10.65,69.41,1.33,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
3,Alimentation Couche Tard (Circle K),82,4.75,8.55,68.03,3.83,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
4,Apple Stores,15,26.65,30.74,11.07,0.25,ElectronicEquipment,0,No rewards program,-,-,-,-,Specialty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Toys ""R"" Us",57,7.53,13.31,79.33,0.87,"Specialty Retail, Other",1,"Rewards""R"" Us",store credit,Spend $125 earn $5 Reward Dollars for future p...,4,12,Specialty
96,Tractor Supply,77,5.17,5.17,2.65,1.28,"Specialty Retail, Other",1,Neighbor's Club,store credit,Spend $150 X3 times earn seasonal (quarterly) ...,1.11,3,Specialty
97,True Value,27,16.33,16.33,38.69,4.49,Home Improvement Stores,1,TrueValue Rewards,store credit,"Spend $250 earn 2,500 points = $5 reward certi...",2,24,Specialty
98,Walgreen,6,68.07,70.10,23.47,8.00,Drug Stores,1,Balance Rewards,store credit,500 points on every prescription filled in the...,10,6,Specialty


In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df.columns

Part I.

Consider logistic regression models with Reward column as the target variable. Fit the model with two indicator variables, one indicating whether a retailer is a discount store (i.e., IndustryType is Discount), and the other indicating whether a retailer is a grocery store (i.e., IndustryType is Grocery). Report the coefficient estimates in the next three questions. [Hint: After you create the dummy variables, use them as Selected Variables (instead of Categorical Variables) in the first step of Logistic Regression.]

## Data Visualization

### Univariate Data Exploration

In [ ]:
df.hist(bins=50, figsize=(20,10))
plt.suptitle('Histogram Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)
plt.tight_layout()
plt.show()

## Data Preprocessing

### Feature Engineering

In [4]:
df["IndustryType"].value_counts()

Specialty      43
Discount       18
Grocery        18
Restaurants    13
Department      8
Name: IndustryType, dtype: int64

In [ ]:
df[(df["IndustryType"] == "Discount") | (df["IndustryType"] == "Grocery")]

In [ ]:
df2 = df[(df["IndustryType"] == "Discount") | (df["IndustryType"] == "Grocery")]

In [ ]:
df2.reset_index(inplace=True, drop=True)

In [ ]:
df2.head()

In [ ]:
df3 = df2[["IndustryType","Reward"]]

In [ ]:
df3.isnull().sum()

In [ ]:
df3

In [ ]:
df3.info()

## Logistic Regression (StatsModel)

In [ ]:
df3.columns

In [ ]:
y = df[['Reward']]
X = df[['IndustryType']]

In [ ]:
X = sm.add_constant(X)

In [ ]:
model = sm.Logit(y, X).fit()

In [ ]:
model.summary()

In [5]:
logitfit = smf.logit(formula = 'Reward ~ C(IndustryType) ', data = df).fit()

Optimization terminated successfully.
         Current function value: 0.647629
         Iterations 6


In [6]:
logitfit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Reward   No. Observations:                  100
Model:                          Logit   Df Residuals:                       95
Method:                           MLE   Df Model:                            4
Date:                Thu, 23 Dec 2021   Pseudo R-squ.:                 0.05887
Time:                        10:37:54   Log-Likelihood:                -64.763
converged:                       True   LL-Null:                       -68.814
Covariance Type:            nonrobust   LLR p-value:                   0.08792
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          1.9459      1.069      1.820      0.069      -0.149       4.041
C(IndustryType)[T.Discount]       -2.3979      1.173     -2.044      0.041      -4.698      -0.098
C(IndustryType)[T.Grocery]        -2.1691      1.170     -1.855      0.064      -4.461       0.123
C(IndustryType)[T.Restaurants]    -2.1001      1.205     -1.743      0.081      -4.462       0.262
C(IndustryType)[T.Specialty]      -1.4227      1.115     -1.276      0.202      -3.607       0.762
==================================================================================================
"""

## Logistic Regression (StatsModel)

In [ ]:
df.columns

In [ ]:
y = df['']
X = df['']

In [ ]:
X = sm.add_constant(X)

In [ ]:
model = sm.Logit(y, X).fit()

In [ ]:
model.summary()

In [ ]:
logitfit = smf.logit(formula = 'DF ~ Debt_Service_Coverage + cash_security_to_curLiab + TNW', data = hgc).fit()

In [ ]:
logitfit = smf.logit(formula = 'DF ~ TNW + C(seg2)', data = hgcdev).fit()

### Logistic Regression (SKLearn)

In [ ]:
df.shape

In [ ]:
X = df.iloc[:,:4]
y = df.iloc[:,4]

In [ ]:
Counter(y)

In [ ]:
X.values, y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=0, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
Counter(y_train), Counter(y_test)

In [ ]:
lr = LogisticRegression(random_state=0)

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
y_pred

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
plot_confusion_matrix(estimator=lr, X=X_test, y_true=y_test, cmap='YlGnBu')
plt.show()

In [ ]:
plot_roc_curve(estimator=lr, X=X_test, y=y_test)
plt.show()

#### Python code done by Dennis Lam